In [ ]:
!pip install lxml
!pip install spacy

In [ ]:
from lxml import etree
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
file_path = '/content/drive/MyDrive/Colab_Notebooks/Commedia_Petrocchi_XML_TEI.xml'

In [ ]:
parser = etree.XMLParser(recover=True)
xml_data = etree.parse(file_path, parser)

In [ ]:
cantiche = xml_data.xpath('//div1')
canti = {}


for cantica_element in cantiche:
    cantica = cantica_element.xpath('./@id')[0]
    tot_canti = cantica_element.xpath('./div2')


    for c in tot_canti:
        h_canto = c.xpath('.//head//text()')
        terzine = []
        terzina = ["".join(l.itertext()).strip() for l in c.xpath('.//l') if "".join(l.itertext()).strip()]

        for i in range(0, len(terzina), 3):
            terzine.append(terzina[i:i+3])

        canti[(cantica, h_canto[0])] = terzine


In [ ]:
data = []
for (cantica, titolo), ter in canti.items():
    i = 1
    for t in ter:
        if len(t) == 3:
            range_versi = f"{i}-{i+2}"
            frase = " ".join(t)
            data.append([cantica, titolo, range_versi, frase])
            i += 3
        else:
            for verso in t:
                range_versi = f"{i}"
                data.append([cantica, titolo, range_versi, verso])
                i += 1


df = pd.DataFrame(data, columns=['cantica', 'canto', 'range_versi', 'terzina'])
df['range_versi'] = "\u200B" + df['range_versi'].astype(str)
df.to_csv("terzine.csv", index=False, sep=';', encoding='utf-8-sig')
#print(df)

In [ ]:
data = []
for (cantica, titolo), ter in canti.items():
    i = 1
    for t in ter:
        for verso in t:
            n_verso = f"{i}"
            data.append([cantica, titolo, n_verso, verso])
            i += 1


df = pd.DataFrame(data, columns=['cantica', 'canto', 'n_verso', 'verso'])
df['n_verso'] = "\u200B" + df['n_verso'].astype(str)
df.to_csv("versi.csv", index=False, sep=';', encoding='utf-8-sig')
#print(df)

In [ ]:
!python -m spacy download it_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 81.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('it_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import spacy

nlp = spacy.load("it_core_news_sm")

canti = {}

for cantica_element in cantiche:
    cantica = cantica_element.xpath('./@id')[0]
    tot_canti = cantica_element.xpath('./div2')

    for c in tot_canti:
        h_canto = c.xpath('.//head//text()')
        versi = ["".join(l.itertext()).strip() for l in c.xpath('.//l') if "".join(l.itertext()).strip()]
        canti[(cantica, h_canto[0])] = versi

data = []

for (cantica, titolo), versi in canti.items():
    testo_canto = " ".join(versi)
    doc = nlp(testo_canto)

    pos_versi = []
    cur_pos = 0
    for v in versi:
        pos_versi.append((cur_pos, cur_pos + len(v)))
        cur_pos += len(v) + 1

    # Analizza ogni frase e trova il range di versi corrispondente
    for frase in doc.sents:
        frase_testo = frase.text.strip()
        if len(frase_testo) <= 2:
            continue  # Ignora frasi troppo corte

        start_idx = testo_canto.find(frase_testo)
        end_idx = start_idx + len(frase_testo)

        versi_inclusi = [
            i + 1 for i, (start, end) in enumerate(pos_versi) if not (end < start_idx or start > end_idx)
        ]

        if not versi_inclusi:
            print(f"Nessun verso trovato per la frase:\n{frase_testo}")
            range_versi = "NA"
        else:
            range_versi = f"{versi_inclusi[0]}-{versi_inclusi[-1]}" if len(versi_inclusi) > 1 else str(versi_inclusi[0])

        data.append([cantica, titolo, range_versi, frase_testo])

df = pd.DataFrame(data, columns=['cantica', 'canto', 'range_versi', 'frase'])
df['range_versi'] = "\u200B" + df['range_versi'].astype(str)
df.to_csv("frasi.csv", index=False, sep=';', encoding='utf-8-sig')

print(df)


       cantica   canto range_versi  \
0      Inferno       I        ​1-3   
1      Inferno       I        ​4-6   
2      Inferno       I          ​7   
3      Inferno       I        ​8-9   
4      Inferno       I      ​10-12   
...        ...     ...         ...   
6100  Paradiso  XXXIII    ​137-138   
6101  Paradiso  XXXIII        ​139   
6102  Paradiso  XXXIII    ​140-141   
6103  Paradiso  XXXIII        ​142   
6104  Paradiso  XXXIII    ​143-145   

                                                  frase  
0     Nel mezzo del cammin di nostra vita mi ritrova...  
1     Ahi quanto a dir qual era è cosa dura esta sel...  
2                    Tant'è amara che poco è più morte;  
3     ma per trattar del ben ch'i' vi trovai, dirò d...  
4     Io non so ben ridir com'i' v'intrai, tant'era ...  
...                                                 ...  
6100  veder voleva come si convenne l'imago al cerch...  
6101               ma non eran da ciò le proprie penne:  
6102  se non che la m

In [ ]:
<?xml version="1.0" encoding="UTF-8"?>
<!DOCTYPE TEI.2 SYSTEM "http://backend.bibliotecaitaliana.it/DTD/bibit.dtd">
<TEI.2 TEIform="TEI.2">
  <teiHeader>
    <fileDesc>
      <titleStmt>
        <title>Commedia</title>
        <author>Dante Alighieri</author>
      </titleStmt>
      <extent>711 Kb in UTF-8</extent>
      <publicationStmt>
        <publisher>Biblioteca Italiana</publisher>
        <pubPlace>Roma</pubPlace>
        <date>2003</date>
        <idno>bibit000019</idno>
        <availability>
          <p>Questa risorsa digitale è liberamente accessibile per uso personale o scientifico. Ogni
            uso commerciale è vietato</p>
        </availability>
      </publicationStmt>
      <seriesStmt>
        <title>Collezione BibIt</title>
      </seriesStmt>
      <sourceDesc>
        <bibl>
          <title>Le opere</title>
          <title type="part">La Commedia secondo l'antica vulgata</title>
          <author>Alighieri, Dante</author>
          <editor id="ed">Societa dantesca italiana</editor>
          <editor id="ed2">Petrocchi, Giorgio</editor>
          <publisher>Mondadori ; [poi] Le Lettere</publisher>
          <pubPlace>Milano ; [poi] Firenze</pubPlace>
          <date>1994</date>
          <note>Edizione nazionale</note>
        </bibl>
      </sourceDesc>
    </fileDesc>
    <encodingDesc>
      <samplingDecl>
        <p>Tutti i materiali paratestuali della fonte cartacea non riconducibili alla responsabilità
          dell'autore dell'opera sono stati soppressi nella versione digitale</p>
      </samplingDecl>
      <editorialDecl>
        <correction method="silent" status="medium">
          <p>Livello medio: controllo a video con collazione con edizione di riferimento</p>
        </correction>
        <quotation form="data" marks="all">
          <p>I simboli di citazione e di discorso diretto presenti sulla fonte cartacea sono stati
            rappresentati sulla versione digitale</p>
        </quotation>
        <hyphenation eol="none">
          <p>I trattini di sillabazione a fine riga sono stati soppressi e le parole ricomposte</p>
        </hyphenation>
      </editorialDecl>
      <classDecl>
        <taxonomy id="CDD">
          <bibl>Classificazione Decimale Dewey</bibl>
        </taxonomy>
        <taxonomy id="CGB">
          <bibl>Classificazione generi BibIt</bibl>
        </taxonomy>
      </classDecl>
    </encodingDesc>
    <profileDesc>
      <creation>
        <date>300</date>
      </creation>
      <langUsage>
        <language id="ita">Italiano</language>
        <language id="lat">Latino</language>
        <language id="occ">Provenzale</language>
      </langUsage>
      <textClass>
        <keywords scheme="CDD">
          <term>851.1 - POESIA ITALIANA, PRIMO PERIODO FINO AL 1375</term>
        </keywords>
        <keywords scheme="CGB">
          <term>Poesia</term>
        </keywords>
      </textClass>
    </profileDesc>
    <revisionDesc>
      <change>
        <date>2000-12-06T00:00:00.000+02:00</date>
        <respStmt>
          <name>Patricia Frosini</name>
          <name>CIBIT-Pisa</name>
        </respStmt>
        <item>Digitalizzazione</item>
      </change>
      <change>
        <date>2000-12-06T00:00:00.000+02:00</date>
        <respStmt>
          <name>Patricia Frosini</name>
          <name>CIBIT-Pisa</name>
        </respStmt>
        <item>Correzione linguistica</item>
      </change>
      <change>
        <date>Data sconosciuta posteriore al 2000</date>
        <respStmt>
          <name>Elena Pierazzo</name>
          <name>CIBIT - Pisa</name>
        </respStmt>
        <item>Codifica XML - Codifica manuale</item>
      </change>
      <change>
        <date>Data sconosciuta posteriore al 2000</date>
        <respStmt>
          <name>Elena Pierazzo</name>
          <name>BIBIT</name>
        </respStmt>
        <item>Validazione</item>
      </change>
      <change>
        <date>Data sconosciuta posteriore al 2000</date>
        <respStmt>
          <name>BIBIT</name>
        </respStmt>
        <item>Revisione linguistica</item>
      </change>
      <change>
        <date>2007-10-11T18:00:00.000+02.00</date>
        <respStmt>
          <name>Fabio Ciotti</name>
        </respStmt>
        <item>Revisione codifica - Inserimento automatico attributo n contenente il numero di verso in ogni elemento l finale di ciascuna terzina
          mediante procedura XSLT</item>
      </change>
    </revisionDesc>
  </teiHeader>
  <text>
    <body lang="ita">
      <div1 id="Inferno" type="parte">
        <head>INFERNO</head>
        <div2 id="Inferno.I" type="canto">
          <head>I</head>
          <lg type="terzina">
            <l>Nel mezzo del cammin di nostra vita</l>
            <l>mi ritrovai per una selva oscura,</l>
            <l n="3">ché la diritta via era smarrita.</l>
          </lg>
          <lg type="terzina">
            <l>Ahi quanto a dir qual era è cosa dura</l>
            <l>esta selva selvaggia e aspra e forte</l>
            <l n="6">che nel pensier rinova la paura!</l>
          </lg>
          <lg type="terzina">
            <l>Tant'è amara che poco è più morte;</l>
            <l>ma per trattar del ben ch'i' vi trovai,</l>
            <l n="9">dirò de l'altre cose ch'i' v'ho scorte.</l>
          </lg>
          <lg type="terzina">
            <l>Io non so ben ridir com'i' v'intrai,</l>
            <l>tant'era pien di sonno a quel punto</l>
            <l n="12">che la verace via abbandonai.</l>
          </lg>
          <lg type="terzina">
            <l>Ma poi ch'i' fui al piè d'un colle giunto,</l>
            <l>là dove terminava quella valle</l>
            <l n="15">che m'avea di paura il cor compunto,</l>
          </lg>
          <lg type="terzina">
            <l>guardai in alto, e vidi le sue spalle</l>
            <l>vestite già de' raggi del pianeta</l>
            <l n="18">che mena dritto altrui per ogne calle.</l>
          </lg>
          <lg type="terzina">
            <l>Allor fu la paura un poco queta,</l>
            <l>che nel lago del cor m'era durata</l>
            <l n="21">la notte ch'i' passai con tanta pieta.</l>
          </lg>
          <lg type="terzina">
            <l>E come quei che con lena affannata,</l>
            <l>uscito fuor del pelago a la riva,</l>
            <l n="24">si volge a l'acqua perigliosa e guata,</l>
          </lg>
          <lg type="terzina">
            <l>così l'animo mio, ch'ancor fuggiva,</l>
            <l>si volse a retro a rimirar lo passo</l>
            <l n="27">che non lasciò già mai persona viva.</l>
          </lg>